# 31 - Linked Lists

---

A linked list is a fairly simple data structure, which is not natively supported by Python, because at first glance the standard Python list seems more powerful. However, for certain problems, linked lists are a more effective solution. This chapter explains how linked lists can be implemented, and how they are useful. An extra benefit is that explaining how linked lists work, will help understanding the next chapters on trees and graphs.

---

## Referencing objects

Objects may reference other objects. You implement this by creating an object and assigning it to an attribute of another object. For instance, below I create a class `Die`, which you can roll, and a class `DiceCup`, which contains a four-sided and six-sided die, which you can use to roll the two dice.

In [ ]:
from random import randint

class Die:
    def __init__( self, sides ):
        self.sides = sides
    def roll( self ):
        return randint( 1, self.sides )
    
class DiceCup:
    def __init__( self ):
        self.foursider = Die( 4 )
        self.sixsider = Die( 6 )
    def roll( self ):
        return self.foursider.roll() + self.sixsider.roll()
    
dicecup = DiceCup()
print( dicecup.roll() )

As you can see, the class `DiceCup` has two attributes, `foursider` and `sixsider`, for which it creates two instances of the class `Die` which it assigns to them. The question is: how are these two dice stored in an instance of `DiceCup`? The answer is: they are "references", or "pointers". What does that mean? It means that the two `Die` objects exist somewhere in the computer's memory, and the `DiceCup` object contains the memory addresses of these two objects. 

So, if I would make a change to one of these `Die` objects, for instance change the number of sides, that affects the results of the `roll()` method of the `DiceCup` object; however, it would not affect the memory addresses that reside inside the `DiceCup` object.

<img src="img/dicecup.png" alt="dicecup" style="width:300px;"><br>
<div align="center">Figure 31.1: Dicecup and two dice.</div>

We say that the `DiceCup` object "links" to two `Die` objects.

### References and values

If you are wondering which data types are stored as references and which as values: the answer is that *all* data types are stored as references (at least in Python 3). However, if an object has an attribute that is of an immutable data type, then even if it got its value by assigning a variable to it, its value will not change if you change the value of the assigned variable. I can imagine that this sounds confusing, so here is an example, whereby the `DiceCup` class has, besides two dice, also a `capacity` which is an integer.

In [ ]:
from random import randint

class Die:
    def __init__( self, sides ):
        self.sides = sides
    def roll( self ):
        return randint( 1, self.sides )
    
class DiceCup:
    def __init__( self ):
        self.foursider = Die( 4 )
        self.sixsider = Die( 6 )
        self.capacity = 0
    def roll( self ):
        return self.foursider.roll() + self.sixsider.roll()
    
dicecup = DiceCup()
capacity = 12
dicecup.capacity = capacity
newdie = Die( 100 )
dicecup.foursider = newdie

capacity = 5
newdie.sides = 5
print( dicecup.capacity )
print( dicecup.foursider.sides )

The last four lines of the code above change the value of the variable `capacity` which was assigned to `dicecup.capacity`, and the value of `newdie.sides`, whereby `newdie` was assigned to `dicecup.foursider`. The printing in the last two lines shows that the capacity of `dicecup` has not changed, but the number of sides of the `foursider` attribute has. This is because `capacity` is an integer and integers are immutable, while `newdie` is an instance of the class `Die`, which is mutable. In principle all instances of classes that are defined by the programmer are mutable.

### Linking objects

Linking objects to each other allows the creation of networks of interlinking objects, which can be used to represent the data structures which underlie numerous problems. One of the simplest is the single-linked list, which will be discussed next.

---

## Single-linked list

The single-linked list is a data structure which consists of a series of `Node` objects, which form a chain. Each node has a reference to the next node in the chain. No node will refer back to an earlier node in the chain, i.e., no cycles will exist in the single-linked list. The last node will refer to `None`. A special `LinkedList` class contains a reference to the first object in the single-linked list.

Below is an implementation of a single-linked list. The `SingleLinkedList` class contains a `head` attribute that points to the first object in the chain. Moreover, it contains a method `add` that adds a new object to the head of the chain, and a `remove` method that removes the head of the sequence. 

The nodes in the sequence are implemented as instances of the `SingleNode` class. They only contain the object that the node represents, and a reference to the next node.

<img src="img/singlelinkedlist.png" alt="single-linked list" style="width:600px;"><br>
<div align="center">Figure 31.2: Single-linked list.</div>

In [ ]:
class SingleNode:
    def __init__( self, value, nextnode ):
        self.value = value
        self.nextnode = nextnode
        
class SingleLinkedList:
    def __init__( self ):
        self.head = None
    def add( self, value ):
        node = SingleNode( value, self.head )
        self.head = node
    def remove( self ):
        if self.head != None:
            self.head = self.head.nextnode
            
sll = SingleLinkedList()
sll.add( 1 )
sll.add( 2 )
sll.add( 3 )

cur = sll.head
while cur != None:
    print( cur.value )
    cur = cur.nextnode

As you can see, traversing the linked list means starting at the head, and then following the chain of `nextnode` references until the end.

Naturally, a better implementation of the `SingleLinkedList` class would include methods to traverse the chain, rather than having the programmer who uses the class access the `head` and `nextnode` attributes in the main code. I will leave that to the students as an exercise, though (see the Exercises section below).

### Efficiency of a single-linked list

You might wonder why you would ever use a single-linked list, which only allows adding new values at the front of the chain, when a regular Python list can do so much more. One reason to use one might be that a single-linked list is much more efficient than a Python list. The reason is that a Python list is implemented as an array (see the chapter on sorting), which is an efficient data structure for looking up items by their index, but a very inefficient data structure for inserting or removing items, unless they reside at the end of the list. This can be demonstrated by comparing two blocks of equivalent code, which add 100,000 numbers at the front of a chain.

The first uses a regular Python list:

In [ ]:
from datetime import datetime

COUNT = 100000
numlist = []

start = datetime.now()

for i in range( COUNT ):
    numlist.insert( 0, i )
    
end = datetime.now()

print( "{}.{} seconds needed to insert {} numbers".format( 
        (end - start).seconds, (end - start).microseconds, COUNT ) )

The second uses the `SingleLinkedList` class defined above.

In [ ]:
from datetime import datetime

class SingleNode:
    def __init__( self, value, nextnode ):
        self.value = value
        self.nextnode = nextnode
        
class SingleLinkedList:
    def __init__( self ):
        self.head = None
    def add( self, value ):
        node = SingleNode( value, self.head )
        self.head = node
    def remove( self ):
        if self.head != None:
            self.head = self.head.nextnode
            
COUNT = 100000

start = datetime.now()

sll = SingleLinkedList()
for i in range( 1, COUNT ):
    sll.add( i )
    
end = datetime.now()

print( "{}.{} seconds needed to insert {} numbers".format( 
        (end - start).seconds, (end - start).microseconds, COUNT ) )

On my computer, the first block of code needs about 2 seconds to insert the 100,000 numbers, while the second needs less than 1. However, when increasing the task to inserting 1,000,000 numbers, the first block needs about 3 *minutes*, while the second is done in just over 1 *second*.

__Exercise__: What is the time complexity of these two blocks of code?

__Answer__: As I discussed in the chapter on sorting, inserting in a Python list is `O(n`<sup>2</sup>`)`. For the single-linked list, insertion of one item needs constant time, since nothing needs to be moved in memory. Therefore, insertion of `n` items is `O(n)`. 

If you wonder why increasing the number of items 10-fold does not require 10 times as much time for the linked list, the answer is that there is always some startup time involved for creating the basic class definitions and setting up the `range()`.

### Appending to a single-linked list

The single-linked list as described above can only add new items at the front of the chain. To allow appending items at the end of the chain, a separate reference to the end of the chain is needed.

In [ ]:
class SingleNode:
    def __init__( self, value, nextnode ):
        self.value = value
        self.nextnode = nextnode
        
class SingleLinkedList:
    def __init__( self ):
        self.head = None
        self.tail = None
    def add( self, value ):
        node = SingleNode( value, self.head )
        self.head = node
        if self.tail == None:
            self.tail = self.head
    def remove( self ):
        if self.head == self.tail:
            self.head = None
            self.tail = None
        else:
            self.head = self.head.nextnode
    def append( self, value ):
        node = SingleNode( value, None )
        if self.tail == None:
            self.head = node
            self.tail = node
        else:
            self.tail.nextnode = node
            self.tail = node
            
sll = SingleLinkedList()
sll.add( 1 )
sll.add( 2 )
sll.add( 3 )
sll.append( 4 )
sll.append( 5 )

cur = sll.head
while cur != None:
    print( cur.value )
    cur = cur.nextnode

Note that adding a reference to the tail of the single-linked list, which allows an efficient implementation of the `append()` method, also impacts the implementation of the other methods. Special care must be taken when adding to an empty chain, as this influences both the head and the tail. Special care must also be taken when removing from a chain with only one item, as this will set both the head and tail to `None`.

Also take note of the fact that in the `append()` method, first the new node gets added to the tail of the chain, and only then the value of `tail` is adapted. It would not work the other way around, because by changing the value of `tail`, I no longer have access to the `nextnode` of the original `tail`.

If besides appending at the end of the list, I also want to allow inserting items in the list, I will need to add at least two methods: one to `find()` a particular item in the chain, after which the new item is to be inserted, and one to `insert()` the new item at that spot in the chain. Insertion, in pseudocode, would be something like:

    spot = sll.find( location )
    node = SingleNode( value, spot.nextnode )
    spot.nextnode = node
    
Of course, I will have to deal with the special cases of having an empty chain, inserting a new node at the start of the list (which will change the head), and inserting a new node after the tail (which will change the tail). Rather than discussing this now, I will bring it up for double-linked lists.

---

## Double-linked list

A double-linked list is a linked list in which each node not only links to the next one, but also to the previous one. The main advantage of a double-linked list is that items can not only be easily added and removed at the head of the list, but also at the tail. Moreover, the linked list can be searched in two directions.

<img src="img/doublelinkedlist.png" alt="double-linked list" style="width:500px;"><br>
<div align="center">Figure 31.3: Double-linked list.</div>

An implementation of a double-linked list is given below. It includes not only adding and removing nodes at the head and tail, but also finding a node with a particular value, and inserting a value after a node. This code is quite long, but mostly straightforward. The main thing to learn from it is that you have to be careful when adding or removing nodes, as this may impact many of the references in the linked list.

In [ ]:
class DoubleNode:
    def __init__( self, value, prevnode, nextnode ):
        self.value = value
        self.prevnode = prevnode
        self.nextnode = nextnode
        
class DoubleLinkedList:
    def __init__( self ):
        self.head = None
        self.tail = None
    def addhead( self, value ):
        node = DoubleNode( value, None, self.head )
        self.head = node
        secondnode = self.head.nextnode
        if secondnode != None:
            secondnode.prevnode = self.head
        else:
            self.tail = self.head
    def addtail( self, value ):
        node = DoubleNode( value, self.tail, None )
        self.tail = node
        penultnode = self.tail.prevnode
        if penultnode != None:
            penultnode.nextnode = self.tail
        else:
            self.head = self.tail
    def removehead( self ):
        if self.head != None:
            self.head = self.head.nextnode
            if self.head != None:
                self.head.prevnode = None
            else:
                self.tail = None
    def removetail( self ):
        if self.tail != None:
            self.tail = self.tail.prevnode
            if self.tail != None:
                self.tail.nextnode = None
            else:
                self.head = None
    def find( self, value ):
        node = self.head
        while node != None:
            if node.value == value:
                return node
            node = node.nextnode
        return None
    def insert( self, node, value ):
        if node == None:
            self.addhead( value )
        elif node.nextnode == None:
            self.addtail( value )
        else:
            newnode = DoubleNode( value, node, node.nextnode )
            node.nextnode = newnode
            newnode.nextnode.prevnode = newnode
            
dll = DoubleLinkedList()
dll.addhead( 2 )
dll.addhead( 1 )
dll.addtail( 3 )
dll.addtail( 4 )

cur = dll.head
while cur != None:
    print( cur.value )
    cur = cur.nextnode
    
for i in range( 6 ):
    if dll.find( i ):
        print( i, "found" )
    else:
        print( i, "not found")
        
node = dll.find( 3 )
dll.insert( node, 10 )

__Exercise__: Add some code to add extra nodes to the double-linked list, remove some nodes, and traverse the double-linked list from tail to head. 

As I said, quite a bit of code is needed to ensure that all the links remain correct. When nodes are added to the head of the list, not only the new node must be created, but the second node also must be updated. When nodes are added to the tail, also the penultimate node must be updated. Special care must be taken to deal with a double-linked list with no nodes or with just one node: any updates affect both the head and the tail. Even more updates must be made when inserting nodes in the linked list.

One point of criticism that can be given with respect to the code is that in principle it is possible to call the `insert()` method with a `node` which is not actually in the double-linked list. There are various ways to solve this. Probably the best is to maintain, next to the `head` and `tail` references, also a `current` reference which is positioned at some node in the double-linked list. Using new methods, the `current` reference can be moved forward and backward through the list. Inserting is only possible after the node that is indicated by `current`. This is similar to how file pointers are used when overwriting parts of a binary file (see the corresponding chapter). I did not add this approach to the code above, as it would make the code even longer as more references must be maintained.

---

## What you learned

In this chapter, you learned about:

- Linking objects
- Single-linked lists
- Double-linked lists

---

## Exercises

### Exercise 31.1

You often want to know the number of items in a linked list. In the code below (which is copied from the chapter), add a method `length()` to `SingleLinkedList` which returns the number of items in the chain. Make sure that it is `O(1)`.

In [ ]:
# count() for SingleLinkedList.

class SingleNode:
    def __init__( self, value, nextnode ):
        self.value = value
        self.nextnode = nextnode
        
class SingleLinkedList:
    def __init__( self ):
        self.head = None
    def add( self, value ):
        node = SingleNode( value, self.head )
        self.head = node
    def remove( self ):
        if self.head != None:
            self.head = self.head.nextnode
            

### Exercise 31.2

In Python, one way to implement a linked list is as a sequence class, as discussed in the chapter on operator overloading.  Implement a single-linked list as a sequence class. Implement `addhead()` and `addtail()` methods to add an item to the start and the end of the linked list, respectively. Implement a `pop()` method to remove the first item of the linked list and return its value. For the `__getitem__()` and `__setitem__()` methods, the key is the index of the item in the linked list. Also implement the `__contains__()` and `__len__()` methods. Implement a `__repr__()` method to display the contents of the linked list. Note that, by using this implementation, you can now use a `for ... in ...` construction to traverse the linked list (as well as using indices).

In [ ]:
# Single-linked list as sequence class.


### Exercise 31.3

An alternative way to set up a linked list is as an iterable object, as discussed in the chapter on iterators and generators. Implement a single-linked list, which allows adding items to the end of the list, as an iterator. I personally prefer to set it up using delegated iteration, as that means that you do not have to reset the iterator explicitly if you wish to traverse it multiple times. If you decide to use delegated iteration, however, make sure that you do not make a copy of the linked list for each delegated object; they should simply consist of a reference to a node in the linked list. 

In [ ]:
# Single-linked list as iterator.


### Exercise 31.4

Create a double-linked list for storing integers. Implement a `sort()` method for this double-linked list.

In [ ]:
# Sorting a double-linked list.


---

End of Chapter 31. Version 1.0. 